In [1]:
from Common_functions import global_features, generate_normal_features,generate_anomaly_features, \
                            get_evaluation_matrix, compute_precision_recall_accuracy
import pandas as pd
from sklearn.model_selection import cross_val_predict, GridSearchCV, ParameterGrid
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import StandardScaler
import pickle


/Users/yury.kasimov/miniconda3/envs/main/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
def read_prev_results(idx):
    return {1 : 'OCSVM_models/FirstDataset/{}_model.pkl',
            2 : 'OCSVM_models/SecondDataset/{}_model.pkl'
           }[idx]

In [4]:
# normal dataset: 1 --- Correspoding anomalous part is 1
#                 2 --- Corresponding anomalous parts are 2 and 3
normal_dataset = 1

# anomalous dataset: 1 --- Emotet malware;  2 --- DarkVNC; 3 --- Simba
anomaly_dataset = 1

# Set to True to run training.
should_train = False


global_features = [
    'clientDestinationPortTotalBytesUDPEstablished',
'clientDestinationPortNumberOfFlowsTCPEstablished',
'clientDestinationPortNumberOfFlowsUDPNotEstablished',
'clientDestinationPortTotalPacketsTCPEstablished',
'clientDestinationPortNumberOfFlowsUDPEstablished',
'clientDestinationPortTotalPacketsTCPNotEstablished',
'clientDestinationPortTotalBytesUDPNotEstablished',
'clientDestinationPortTotalBytesTCPEstablished',
'clientDestinationPortTotalPacketsUDPNotEstablished',
'clientDestinationPortNumberOfFlowsTCPNotEstablished',
'clientDestinationPortTotalBytesTCPNotEstablished',
'clientDestinationPortTotalPacketsUDPEstablished']

feature_abbrv = {k:''.join([c for c in k if c.isupper()]) for k in global_features }
scalers = {}

In [5]:
# read models.
models = {}
if not should_train:
    models_folder = read_prev_results(normal_dataset)
    for feature in global_features:       
        with open(models_folder.format(feature), 'rb') as handle:
            models[feature] = pickle.load(handle)
    
    


/Users/yury.kasimov/miniconda3/envs/main/lib/python3.6/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator OneClassSVM from version 0.19.0 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [6]:
def get_split_data(feature_name, normal_dataset, anomaly_dataset):
    features = generate_normal_features(feature_name, dataset=normal_dataset)    
    transformed_anomaly = generate_anomaly_features(feature_name,dataset=anomaly_dataset)

    X_train, X_test, labels_train, labels_test = train_test_split(features, [1]*len(features), test_size=0.2, random_state=42)
    X_train, X_val, labels_train, labels_val = train_test_split(features, [1]*len(features), test_size=0.25, random_state=42)

    np.random.seed(42)
    idx= np.random.choice(range(0,transformed_anomaly.shape[0]),int(transformed_anomaly.shape[0]/2), replace=False)
    validation_anomalies = transformed_anomaly[idx,:]
    idx = [x for x in range(transformed_anomaly.shape[0]) if x in set(idx)]
    test_anomaly = transformed_anomaly[idx,:]

    X_val = np.append(X_val, validation_anomalies, axis=0)
    X_test = np.append(X_test, test_anomaly, axis=0)
    
    labels_val = np.append(np.array(labels_val), np.array([-1]*int(transformed_anomaly.shape[0]/2)))
    labels_test = np.append(np.array(labels_test), np.array([-1]*int(transformed_anomaly.shape[0]/2)))
    benign_val = range(0, labels_val.shape[0]-int(transformed_anomaly.shape[0]/2))
    anomaly_val = range(labels_val.shape[0]-int(transformed_anomaly.shape[0]/2), labels_val.shape[0])
    benign_test = range(0, labels_test.shape[0]-int(transformed_anomaly.shape[0]/2))
    anomaly_test = range(labels_test.shape[0]-int(transformed_anomaly.shape[0]/2), labels_test.shape[0])
    
    
    return (X_train, labels_train), (X_val, labels_val), (X_test, labels_test), (benign_val, anomaly_val), (benign_test, anomaly_test)

## Grid search to find the best models for each feature

In [7]:
if should_train:
    scores = {}

    should_scale = True
    rs = np.random.RandomState(42)
    parameters = {'gamma' : np.logspace(-9, 3, 13), 'nu' : np.linspace(0.01, 0.99, 99)}

    feature_abbrv = {k:''.join([c for c in k if c.isupper()]) for k in global_features }
    dataframe_columns = list(feature_abbrv.values())
    experiment_results = pd.DataFrame(columns=['parameters','evaluation']+dataframe_columns)
    experiment_results.set_index(['parameters','evaluation'], inplace=True)

    test_anomalies = {}
    test_labels = {}
    for z in ParameterGrid(parameters):
        kernel_string = ', '.join('{} : {}'.format(key, val) for key, val in z.items())
        print(z)
        for feature_name in global_features:       

            features = generate_normal_features(feature_name, dataset=normal_dataset)    
            transformed_anomaly = generate_anomaly_features(feature_name,dataset=anomaly_dataset)
            labels_anomaly = np.array([-1]*transformed_anomaly.shape[0])
            X_train, X_test, labels_train, labels_test = train_test_split(features, [1]*len(features), test_size=0.2, random_state=42)
            X_train, X_val, labels_train, labels_val = train_test_split(features, [1]*len(features), test_size=0.25, random_state=42)

            if should_scale:
                scaler = scalers.get(feature_name, StandardScaler(with_std=True, with_mean=True).fit(X_train))
                scalers[feature_name] = scaler
                X_train = scaler.transform(X_train)



            np.random.seed(42)
            idx= np.random.choice(range(0,transformed_anomaly.shape[0]),int(transformed_anomaly.shape[0]/2), replace=False)
            validation_anomalies = transformed_anomaly[idx,:]
            idx = [x for x in range(transformed_anomaly.shape[0]) if x in set(idx)]
            test_anomaly = transformed_anomaly[idx,:]
            test_anomalies[feature_name] = test_anomaly

            X_val = np.append(X_val, validation_anomalies, axis=0)
            if should_scale:
                X_val = scaler.transform(X_val)
            labels_val = np.append(np.array(labels_val), np.array([-1]*int(transformed_anomaly.shape[0]/2)))
            benign_val = range(0, labels_val.shape[0]-int(transformed_anomaly.shape[0]/2))
            anomaly_val = range(labels_val.shape[0]-int(transformed_anomaly.shape[0]/2), labels_val.shape[0])

            svm = OneClassSVM()
            svm.set_params(**z)
            svm.fit(X_train)
            predicted = svm.predict(X_val)

            true_positive, false_positive, true_negative, false_negative = \
                                                get_evaluation_matrix(labels=labels_val, predicted=predicted, 
                                                          benign_range=benign_val, anomaly_range=anomaly_val)


            precision, recall, accuracy = compute_precision_recall_accuracy(true_positive=true_positive,
                                                                            true_negative=true_negative,
                                                                            false_positive=false_positive,
                                                                            false_negative=false_negative)
            experiment_results.loc[(kernel_string,'tp'), feature_abbrv[feature_name]]=true_positive
            experiment_results.loc[(kernel_string,'fp'), feature_abbrv[feature_name]]=false_positive
            experiment_results.loc[(kernel_string,'tn'), feature_abbrv[feature_name]]=true_negative
            experiment_results.loc[(kernel_string,'fn'), feature_abbrv[feature_name]]=false_negative
            experiment_results.loc[(kernel_string,'precision'), feature_abbrv[feature_name]]=precision
            experiment_results.loc[(kernel_string,'recall'), feature_abbrv[feature_name]]=recall
            experiment_results.loc[(kernel_string,'accuracy'), feature_abbrv[feature_name]]=accuracy
            experiment_results.loc[(kernel_string,'FPR'), feature_abbrv[feature_name]]=false_positive/(false_positive+true_negative)
            experiment_results.loc[(kernel_string,'TPR'), feature_abbrv[feature_name]]=true_positive/(true_positive+false_negative)


## Selecting the best parameters based on the results from grid search



In [8]:
if should_train:
    model_params = {k : {} for k in global_features}
    min_tpr = 0.167 # 0.167*6 = 1.002 it means we will detect an attack during first 30 minutes
    for feature, abriv in feature_abbrv.items():
        tmp = experiment_results.unstack(1)[abriv]
        tpr_max_val_fpr_less_001 = tmp[tmp['FPR'] < 0.01]['TPR'].max()
        tpr_max_val_fpr_min = tmp[tmp['FPR'] == tmp['FPR'].min()]['TPR'].max()
        print('=====================\n',feature)
        if tpr_max_val_fpr_min> min_tpr:
            print(tmp[tmp['FPR'] == tmp['FPR'].min()][['FPR','TPR', 'precision', 'recall']])
            params = tmp[tmp['FPR'] == tmp['FPR'].min()]['TPR'].argmax().split(', ')
        elif tpr_max_val_fpr_less_001 > tpr_max_val_fpr_min:
            print(tmp[tmp['FPR'] < 0.01][['FPR','TPR','precision', 'recall']])
            params = tmp[tmp['FPR'] < 0.01]['TPR'].argmax().split(', ')
        else:
            print(tmp[tmp['FPR'] == tmp['FPR'].min()][['FPR','TPR','precision', 'recall']])
            params = tmp[tmp['FPR'] == tmp['FPR'].min()]['TPR'].argmax().split(', ')
        for p in params:
            p = p.split(' : ')
            p_name = p[0]
            p_value = float(p[1])
            model_params[feature][p_name] = p_value
            
    for feature in global_features:
        print(feature, model_params[feature])

## Ensemble

## Evaluate models for each individual feature

In [9]:
should_scale=True
predictions = []
for feature in global_features:

    (X_train, labels_train), \
    (X_val, labels_val), \
    (X_test, labels_test), \
    (benign_val, anomaly_val), \
    (benign_test, anomaly_test) = get_split_data(feature, normal_dataset, anomaly_dataset)
    
    if should_scale:
        if feature in scalers:
            sc = scalers[feature]
        else: 
            sc = StandardScaler(with_std=True, with_mean=True).fit(X_train)
        X_train = sc.transform(X_train)
        X_test = sc.transform(X_test)
        X_val = sc.transform(X_val)
    
    svm = models[feature]
    predicted = svm.predict(X_test)
    predictions.append(predicted)

    true_positive, false_positive, true_negative, false_negative = \
                                            get_evaluation_matrix(labels=labels_test, predicted=predicted, 
                                                      benign_range=benign_test, anomaly_range=anomaly_test)
    
    precision, recall, accuracy = compute_precision_recall_accuracy(true_positive=true_positive,
                                                                        true_negative=true_negative,
                                                                        false_positive=false_positive,
                                                                        false_negative=false_negative)
    fpr = false_positive/(false_positive+true_negative)
    tpr = true_positive/(true_positive+false_negative)
    
    print(feature)
    print('fpr: ',fpr,'tpr: ', tpr, 'precision: ',precision)
    print('========================')


clientDestinationPortTotalBytesUDPEstablished
fpr:  0.0 tpr:  0.3888888888888889 precision:  1.0


/Users/yury.kasimov/PycharmProjects/AnomalyDetectionProfiles/source/AnomalyDetection/Common_functions.py:128: RuntimeWarning: invalid value encountered in long_scalars
  precision = true_positive/(true_positive + false_positive)


clientDestinationPortNumberOfFlowsTCPEstablished
fpr:  0.0 tpr:  0.0 precision:  nan
clientDestinationPortNumberOfFlowsUDPNotEstablished
fpr:  0.0 tpr:  0.3888888888888889 precision:  1.0
clientDestinationPortTotalPacketsTCPEstablished
fpr:  0.01098901098901099 tpr:  1.0 precision:  0.9473684210526315
clientDestinationPortNumberOfFlowsUDPEstablished
fpr:  0.01098901098901099 tpr:  0.3888888888888889 precision:  0.875
clientDestinationPortTotalPacketsTCPNotEstablished
fpr:  0.0 tpr:  1.0 precision:  1.0
clientDestinationPortTotalBytesUDPNotEstablished
fpr:  0.01098901098901099 tpr:  0.3888888888888889 precision:  0.875
clientDestinationPortTotalBytesTCPEstablished
fpr:  0.01098901098901099 tpr:  1.0 precision:  0.9473684210526315
clientDestinationPortTotalPacketsUDPNotEstablished
fpr:  0.0 tpr:  0.3888888888888889 precision:  1.0


/Users/yury.kasimov/miniconda3/envs/main/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


clientDestinationPortNumberOfFlowsTCPNotEstablished
fpr:  0.0 tpr:  1.0 precision:  1.0
clientDestinationPortTotalBytesTCPNotEstablished
fpr:  0.0 tpr:  1.0 precision:  1.0
clientDestinationPortTotalPacketsUDPEstablished
fpr:  0.0 tpr:  0.3888888888888889 precision:  1.0


### Evaluate the majority voting

In [10]:
majority_voting = sum(predictions)
majority_voting[majority_voting>0] = 1
majority_voting[majority_voting<=0] = -1


true_positive, false_positive, true_negative, false_negative = \
                                            get_evaluation_matrix(labels=labels_test, predicted=majority_voting, 
                                                      benign_range=benign_test, anomaly_range=anomaly_test)

fpr = false_positive/(false_positive+true_negative)
tpr = true_positive/(true_positive+false_negative)
print('FPR ensemble: {}\nTPR ensebmle: {}'.format(fpr, tpr))

FPR ensemble: 0.0
TPR ensebmle: 0.4444444444444444


### Save models

In [ ]:
for feature, model in models.items():
    with open(models_folder.format(feature).format(feature), 'wb') as f:
        pickle.dump(model, f, protocol=pickle.HIGHEST_PROTOCOL)